In [23]:
from typing import List, Set, Tuple
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [ ]:
excel_path  = r"C:\Users\issam\Desktop\Files\PhD\DRPRL\data\StaticProblemInstances\GroupA\distancematrix_with_20_nodes(seconds).xlsx"
D1 = pd.read_excel(excel_path, sheet_name='D1', header = None).values
battery_capacity = 1800


Model

In [ ]:
model = gp.Model("DRP")
model.setParam('TimeLimit', 300)

Variables

In [12]:
n = len(D1)
x = {}
for i in range(n):
    for j in range(n):  
        if i != j:
            x[i,j] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')

In [ ]:
# Print variable count
print(f"Number of variables: {len(x)}")

objective function 

In [14]:
obj = gp.quicksum(D1[i,j]*x[i,j] for i,j in x)
model.setObjective(obj, GRB.MINIMIZE)

constraints <br> flow conservation

In [15]:
for h in range(20):
    model.addConstr(gp.quicksum(x[i,h] for i in range(n) if i != h) == 
                    gp.quicksum(x[h,j] for j in range(n) if h != j))

visit each customer once 

In [16]:
for h in range(20) :
    model.addConstr(gp.quicksum(x[i,j] for i in range(n) if i!=j) == 1)

Battery constraints


In [17]:
V = set(range(n))
battery_var = model.addVars(V, vtype=GRB.CONTINUOUS, name='battery')

for i,j in x:
    if D1[i,j] > battery_capacity:
        model.addConstr(x[i,j] == 0)

for i,j in x : 
    model.addConstr(battery_var[j] <= battery_var[i] - D1[i,j] * x[i,j])

# Add stations recharging
stations = set(range(20, 25))  # Last 5 nodes are stations
for i,j in x:
    if j in stations:  # If arriving at station
        model.addConstr(battery_var[j] == battery_capacity)  # Full recharge

Solve

In [ ]:
model.optimize()

In [ ]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective: {model.objVal}")
    print("\nSelected arcs:")
    for i,j in x:
        if x[i,j].X > 0.5:
            print(f"{i} -> {j} (cost: {D1[i,j]})")
else: 
    print("No solution found")

In [ ]:
!python -m pip install -U matplotlib

In [22]:
import matplotlib.pyplot as plt